In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text

## import data
We start by importing the desiered dataset from the tensorflow resources.

In [52]:
batch_size = 32
train = tfds.load('ag_news_subset', split='train', shuffle_files=True, as_supervised = True, batch_size=batch_size)
test = tfds.load('ag_news_subset', split='test', shuffle_files=True, as_supervised = True, batch_size=batch_size)

## import bert

We import a light bert from tesnorflow hub to later be finetuned. We can also import the acompanied preprocessing layer.

In [94]:
bert_preprocessing = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3', name='preprocessing')
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2', trainable=False, name = 'BERT')

## Build the model
We create a function to define and compile the NN with the pretrained bert model

In [ ]:
def build_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='description')
  encoder_inputs = bert_preprocessing(text_input)
  outputs = bert(encoder_inputs)
  
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(32, activation=None)(net)
  net = tf.keras.layers.Dense(16, activation=None)(net)
  net = tf.keras.layers.Dense(4, activation=None)(net)
  
  model =tf.keras.Model(text_input, net)
  model.compile(optimizer = 'Adam', loss = 'SparseCategoricalCrossentropy', metrics = ['accuracy'])

  return model

Build and summarize the model

In [ ]:
model = build_model()
model.summary()

Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
description (InputLayer)        [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_mask': (None 0           description[0][0]                
__________________________________________________________________________________________________
BERT (KerasLayer)               {'default': (None, 1 4385921     preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
___________________________________________________________________________________________

Now that the model in compiled, we can train on our data. We use early stopping to prevent overfitting

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)

history = model.fit(
        x = train,
        validation_data=test,
        epochs=1,
        callbacks=[earlystopping],
        batch_size=32,
        verbose=1)

 107/3750 [..............................] - ETA: 11:16 - loss: 2.0696 - accuracy: 0.2678

KeyboardInterrupt: 